# Data Sampling and Transformation from CICFlowMeter Dataset
## Part 1: Selecting specific columns and sampling based on Label values

-----------

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os

In [ ]:
# Read data
df=pd.read_csv(r'E:\projects ML\Network_attack\Data\CICFlowMeter.csv')

In [ ]:
# to see first 5 rows
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,175.45.176.2-149.171.126.16-23357-80-6,175.45.176.2,23357,149.171.126.16,80,6,22/01/2015 07:50:15 AM,214392,9,21,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Exploits
1,175.45.176.0-149.171.126.16-13284-80-6,175.45.176.0,13284,149.171.126.16,80,6,22/01/2015 07:50:13 AM,2376792,9,3,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Reconnaissance
2,175.45.176.2-149.171.126.16-13792-5555-6,175.45.176.2,13792,149.171.126.16,5555,6,22/01/2015 07:50:16 AM,131350,10,3,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Exploits
3,175.45.176.0-149.171.126.15-39500-80-6,175.45.176.0,39500,149.171.126.15,80,6,22/01/2015 07:50:18 AM,164796,6,3,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS
4,175.45.176.0-149.171.126.14-29309-3000-6,175.45.176.0,29309,149.171.126.14,3000,6,22/01/2015 07:50:19 AM,163418,6,3,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Generic


In [7]:
# Required columns
selected_columns = [
    'Flow Duration',
    'Protocol',
    'Src Port',
    'Dst Port',
    'Total Length of Fwd Packet',
    'Total Length of Bwd Packet',
    'Total Fwd Packet',
    'Total Bwd packets',
    'Flow Bytes/s',
    'Flow Packets/s',
    'Packet Length Mean',
    'Packet Length Std',
    'Label'
]

# Make sure all the columns are present
missing_columns = [col for col in selected_columns if col not in df.columns]
if missing_columns:
    print(f"Warning: These columns are missing: {missing_columns}")
    print(f"\nAvailable columns in dataset:")
    print(df.columns.tolist())
else:
    print("✅ All required columns are present!")

✅ All required columns are present!


In [8]:
# Select only the required columns
df_selected = df[selected_columns].copy()
print(f"Dataset after column selection: {df_selected.shape}")

Dataset after column selection: (3540241, 13)


In [9]:
# Labels
target_labels = ['Benign', 'Exploits', 'Fuzzers', 'Reconnaissance', 'Generic', 'DoS', 'Shellcode']

# Create a list to store samples
sampled_data = []

# Process each label
for label in target_labels:
    # Filter data by label
    label_data = df_selected[df_selected['Label'] == label]
    
    if label == 'Benign':
        # Only 10,000 rows were taken from Benign
        if len(label_data) >= 10000:
            sampled_label_data = label_data.sample(n=10000, random_state=42)
            print(f"{label}: Sampled 10000 rows from {len(label_data)} available rows")
        else:
            sampled_label_data = label_data
            print(f"{label}: Only {len(label_data)} rows available (less than 10000)")
    else:
        # Take all the rows of the remaining labels
        sampled_label_data = label_data
        print(f"{label}: Taking all {len(label_data)} rows")
    
    sampled_data.append(sampled_label_data)

# Merge all samples into a single dataframe
final_df = pd.concat(sampled_data, ignore_index=True)

print(f"\n{'='*50}")
print(f"Final dataset shape: {final_df.shape}")
print(f"\nFinal Label distribution:")
print(final_df['Label'].value_counts())

Benign: Sampled 10000 rows from 3450658 available rows
Exploits: Taking all 30951 rows
Fuzzers: Taking all 29613 rows
Reconnaissance: Taking all 16735 rows
Generic: Taking all 4632 rows
DoS: Taking all 4467 rows
Shellcode: Taking all 2102 rows

Final dataset shape: (98500, 13)

Final Label distribution:
Label
Exploits          30951
Fuzzers           29613
Reconnaissance    16735
Benign            10000
Generic            4632
DoS                4467
Shellcode          2102
Name: count, dtype: int64


## Part 2: Protocol Encoding and Column Renaming

In [10]:
# Check the existing protocol values

print("Protocol values in dataset:")
print(final_df['Protocol'].value_counts())
print(f"\nUnique Protocol values: {final_df['Protocol'].unique()}")

Protocol values in dataset:
Protocol
6     83761
17    14739
Name: count, dtype: int64

Unique Protocol values: [ 6 17]


In [11]:
# Convert Protocol to two columns: protocol_tcp and protocol_udp
# Protocol = 6 means TCP
# Protocol = 17 means UDP

final_df['protocol_tcp'] = (final_df['Protocol'] == 6).astype(int)
final_df['protocol_udp'] = (final_df['Protocol'] == 17).astype(int)

print("Protocol encoding completed!")
print(f"\nprotocol_tcp distribution:")
print(final_df['protocol_tcp'].value_counts())
print(f"\nprotocol_udp distribution:")
print(final_df['protocol_udp'].value_counts())

# Delete the original Protocol column
final_df = final_df.drop('Protocol', axis=1)
print("\n✅ Original Protocol column removed")

Protocol encoding completed!

protocol_tcp distribution:
protocol_tcp
1    83761
0    14739
Name: count, dtype: int64

protocol_udp distribution:
protocol_udp
0    83761
1    14739
Name: count, dtype: int64

✅ Original Protocol column removed


In [14]:
# Rename the columns according to the required mapping.
column_mapping = {
    'Flow Duration': 'duration',
    'Src Port': 'src_port',
    'Dst Port': 'dst_port',
    'Total Length of Fwd Packet': 'orig_bytes',
    'Total Length of Bwd Packet': 'resp_bytes',
    'Total Fwd Packet': 'orig_pkts',
    'Total Bwd packets': 'resp_pkts',
    'Flow Bytes/s': 'bytes_per_second',
    'Flow Packets/s': 'packets_per_second',
    'Packet Length Mean': 'packet_length_mean',
    'Packet Length Std': 'packet_length_std',
    'Label': 'label'
}

# Rename application
final_df = final_df.rename(columns=column_mapping)

print("✅ Column renaming completed!")
print(f"\nNew column names:")
for i, col in enumerate(final_df.columns, 1):
    print(f"{i:2d}. {col}")

✅ Column renaming completed!

New column names:
 1. duration
 2. src_port
 3. dst_port
 4. orig_bytes
 5. resp_bytes
 6. orig_pkts
 7. resp_pkts
 8. bytes_per_second
 9. packets_per_second
10. packet_length_mean
11. packet_length_std
12. label
13. protocol_tcp
14. protocol_udp


In [15]:
# Reorder the columns in the desired order
desired_order = [
    'duration',
    'protocol_tcp',
    'protocol_udp',
    'src_port',
    'dst_port',
    'orig_bytes',
    'resp_bytes',
    'orig_pkts',
    'resp_pkts',
    'bytes_per_second',
    'packets_per_second',
    'packet_length_mean',
    'packet_length_std',
    'label'
]

final_df = final_df[desired_order]

print("✅ Columns reordered successfully!")
print(f"\nFinal dataset shape: {final_df.shape}")
print(f"\nFinal columns in order:")
for i, col in enumerate(final_df.columns, 1):
    print(f"{i:2d}. {col}")

✅ Columns reordered successfully!

Final dataset shape: (98500, 14)

Final columns in order:
 1. duration
 2. protocol_tcp
 3. protocol_udp
 4. src_port
 5. dst_port
 6. orig_bytes
 7. resp_bytes
 8. orig_pkts
 9. resp_pkts
10. bytes_per_second
11. packets_per_second
12. packet_length_mean
13. packet_length_std
14. label


In [17]:
# Save the final file

output_file = '../Data/CICFlowMeter_processed.csv'
final_df.to_csv(output_file, index=False)
print(f"✅ Data saved successfully to: {output_file}")
print(f"File size: {final_df.shape[0]} rows × {final_df.shape[1]} columns")

✅ Data saved successfully to: ../Data/CICFlowMeter_processed.csv
File size: 98500 rows × 14 columns


In [ ]:
# to know shape data
final_df.shape

(98500, 14)

In [ ]:
# to see first 5 rows
final_df.head()

,duration,protocol_tcp,protocol_udp,src_port,dst_port,orig_bytes,resp_bytes,orig_pkts,resp_pkts,bytes_per_second,packets_per_second,packet_length_mean,packet_length_std,label
0,26672,1,0,52646,29639,486.0,42190.0,58,59,1.600030e+06,4386.622675,361.661017,558.736383,Benign
1,304,1,0,36817,6881,0.0,0.0,1,2,0.000000e+00,9868.421053,0.000000,0.000000,Benign
2,1103,0,1,46139,53,90.0,122.0,2,2,1.922031e+05,3626.473255,51.400000,8.763561,Benign
3,353,1,0,65133,1144,0.0,0.0,1,2,0.000000e+00,8498.583569,0.000000,0.000000,Benign
4,571137,1,0,56407,6881,1278.0,525432.0,220,435,9.222131e+05,1146.835173,802.911585,701.702106,Benign


In [ ]:
#information about the dataset
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98500 entries, 0 to 98499
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   duration            98500 non-null  int64  
 1   protocol_tcp        98500 non-null  int64  
 2   protocol_udp        98500 non-null  int64  
 3   src_port            98500 non-null  int64  
 4   dst_port            98500 non-null  int64  
 5   orig_bytes          98500 non-null  float64
 6   resp_bytes          98500 non-null  float64
 7   orig_pkts           98500 non-null  int64  
 8   resp_pkts           98500 non-null  int64  
 9   bytes_per_second    98500 non-null  float64
 10  packets_per_second  98500 non-null  float64
 11  packet_length_mean  98500 non-null  float64
 12  packet_length_std   98500 non-null  float64
 13  label               98500 non-null  object 
dtypes: float64(6), int64(7), object(1)
memory usage: 10.5+ MB


--------

## Part3 : Merge datasets

## Read data

In [20]:
BASE_DIR= os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(BASE_DIR, 'Data','zeek_attack_10000.csv')
df1 = pd.read_csv(DATA_DIR)

## `EDA`

In [ ]:
# to know shape data
df1.shape

(10000, 14)

In [ ]:
# to see first 5 rows
df1.head()

,duration,protocol_tcp,protocol_udp,src_port,dst_port,orig_bytes,resp_bytes,orig_pkts,resp_pkts,bytes_per_second,packets_per_second,packet_length_mean,packet_length_std,label
0,0.048474,0,1,62187,53,48.091995,181.291048,0.909166,1.017716,13851.433234,114.929616,107.427817,11.658006,Fileless_Macro_Attack
1,0.037844,0,1,51973,53,42.625596,395.150513,1.051842,1.143929,6260.962767,28.041209,208.291741,18.901093,Fileless_Macro_Attack
2,0.000000,0,1,58910,53,61.909705,191.515089,1.007395,0.989098,5709.583294,42.632804,149.152652,12.826899,Fileless_Macro_Attack
3,0.000000,0,1,138,138,0.053518,0.000000,1.019477,0.024698,0.009581,0.000000,0.057565,0.020983,Fileless_Macro_Attack
4,161.498428,0,0,3,3,2642.377757,0.105117,10.679325,0.026896,16.182921,0.055924,226.863793,24.541171,Fileless_Macro_Attack


In [ ]:
# Check the distribution of labels
df1['label'].value_counts()

label
Fileless_Macro_Attack    10000
Name: count, dtype: int64

In [ ]:
# Check column compatibility
print("Checking column compatibility...")
print(f"\nfinal_df columns: {final_df.columns.tolist()}")
print(f"\nzeek_df columns: {df1.columns.tolist()}")

# Check the match
if list(final_df.columns) == list(df1.columns):
    print("\n✅ Columns are identical! Ready to merge.")
else:
    print("\n⚠️ Warning: Columns are different!")
    missing_in_zeek = set(final_df.columns) - set(df1.columns)
    missing_in_final = set(df1.columns) - set(final_df.columns)
    if missing_in_zeek:
        print(f"Missing in zeek_df: {missing_in_zeek}")
    if missing_in_final:
        print(f"Missing in final_df: {missing_in_final}")

Checking column compatibility...

final_df columns: ['duration', 'protocol_tcp', 'protocol_udp', 'src_port', 'dst_port', 'orig_bytes', 'resp_bytes', 'orig_pkts', 'resp_pkts', 'bytes_per_second', 'packets_per_second', 'packet_length_mean', 'packet_length_std', 'label']

zeek_df columns: ['duration', 'protocol_tcp', 'protocol_udp', 'src_port', 'dst_port', 'orig_bytes', 'resp_bytes', 'orig_pkts', 'resp_pkts', 'bytes_per_second', 'packets_per_second', 'packet_length_mean', 'packet_length_std', 'label']

✅ Columns are identical! Ready to merge.


In [ ]:
# Merging datasets
print("Merging datasets...")
merged_df = pd.concat([final_df, df1], ignore_index=True)

print(f"\n{'='*60}")
print(f"✅ Merge completed successfully!")
print(f"{'='*60}")
print(f"\nfinal_df shape: {final_df.shape}")
print(f"zeek_df shape: {df1.shape}")
print(f"merged_df shape: {merged_df.shape}")
print(f"\nTotal rows: {final_df.shape[0]} + {df1.shape[0]} = {merged_df.shape[0]}")

Merging datasets...

✅ Merge completed successfully!

final_df shape: (98500, 14)
zeek_df shape: (10000, 14)
merged_df shape: (108500, 14)

Total rows: 98500 + 10000 = 108500


In [ ]:
# Display information about the combined data
print("Merged Dataset Label Distribution:")
print(merged_df['label'].value_counts())


Merged Dataset Label Distribution:
label
Exploits                 30951
Fuzzers                  29613
Reconnaissance           16735
Benign                   10000
Fileless_Macro_Attack    10000
Generic                   4632
DoS                       4467
Shellcode                 2102
Name: count, dtype: int64


In [ ]:
# Save the final merged file
output_file_merged = '../Data/CICFlowMeter_merged_final.csv'
merged_df.to_csv(output_file_merged, index=False)
print(f"✅ Merged data saved successfully to: {output_file_merged}")
print(f"File size: {merged_df.shape[0]} rows × {merged_df.shape[1]} columns")

✅ Merged data saved successfully to: ../Data/CICFlowMeter_merged_final.csv
File size: 108500 rows × 14 columns
